In [3]:
import json

import openai
from openai import OpenAI

import os
import sys

from dotenv import load_dotenv
load_dotenv('../../.env')

from elasticsearch import Elasticsearch

from tqdm.auto import tqdm

In [4]:
# Obtener la ruta absoluta de la carpeta 'intro'
intro_path = os.path.abspath(os.path.join('..'))
# Agregar la carpeta 'intro' al sys.path
sys.path.insert(0, intro_path)
# Ahora puedes importar minsearch
import minsearch

In [8]:
OPENAI_API_KEY: str = os.getenv('API_KEY')